In [1]:
%cd ..

/home/nikita/edu/ai-masters/nla1/project


In [2]:
from typing import Union
from tqdm import tqdm
from collections import deque
from statistics import mean
from functools import partial

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

from torchvision.models import alexnet, AlexNet_Weights

from conv_cp.conv_cp import decompose_model
from conv_cp.imagenet.dataset import ImageNet

In [3]:
def acc_1(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    y_pred = y_pred.argmax(dim=1)
    correct = (y_pred == y_true).sum().item()
    return correct / y_true.size(0)


def acc_5(y_pred: torch.Tensor, y_true: torch.Tensor) -> float:
    y_pred = y_pred.topk(5, dim=1).indices
    correct = (y_pred == y_true.unsqueeze(1)).sum().item()
    return correct / y_true.size(0)


def loss_fn(
    model: nn.Module,
    dataloader: DataLoader,
    device: Union[str, torch.device],
    verbose: bool = False,
    num_steps: int = 50,
) -> float:
    model.eval()
    model.to(device)

    total_loss = 0
    data_iter = iter(dataloader)
    loop = range(num_steps)
    if verbose:
        loop = tqdm(range(num_steps), desc="Validation")
    for step in loop:
        try:
            x, y = next(data_iter)
        except StopIteration:
            break
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            y_pred = model(x)
            loss = F.cross_entropy(y_pred, y)
            total_loss += loss.item()

        if verbose:
            loop.set_postfix(loss=total_loss / (step + 1))

    model.cpu()
    return total_loss / num_steps


def train_fn(
    model: nn.Module,
    dataloader: DataLoader,
    device: Union[str, torch.device],
    lr: float,
    num_steps: int = 1000,
    metric_len: int = 20,
    loss_tol: float = 1e-3,
    acc_tol: float = 0.95,
    verbose: bool = False,
):
    model.train()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    running_loss = deque(maxlen=metric_len)
    running_acc = deque(maxlen=metric_len)
    runngin_acc_5 = deque(maxlen=metric_len)

    data_iter = iter(dataloader)
    loop = range(num_steps)
    if verbose:
        loop = tqdm(range(num_steps), desc="Training")
    for _ in loop:
        try:
            x, y = next(data_iter)
        except StopIteration:
            data_iter = iter(dataloader)
            x, y = next(data_iter)

        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred = model(x)
        loss = F.cross_entropy(y_pred, y)
        loss.backward()
        optimizer.step()

        running_loss.append(loss.item())
        running_acc.append(acc_1(y_pred, y))
        runngin_acc_5.append(acc_5(y_pred, y))

        if verbose:
            loop.set_postfix(
                loss=mean(running_loss),
                acc_1=mean(running_acc),
                acc_5=mean(runngin_acc_5),
            )

        if mean(running_loss) < loss_tol or mean(running_acc) > acc_tol:
            break

    optimizer.zero_grad()
    model.cpu()
    model.eval()

In [4]:
model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
transform = AlexNet_Weights.IMAGENET1K_V1.transforms()

dataset = ImageNet(root_dir="data/val-images", transform=transform)
train_dataset, val_dataset = dataset.split(0.9)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [5]:
loss_fn = partial(
    loss_fn,
    dataloader=val_loader,
    device="cuda",
    num_steps=50,
    verbose=True,
)
train_fn = partial(
    train_fn,
    dataloader=train_loader,
    device="cuda",
    lr=1e-7,
    metric_len=50,
    num_steps=500,
    verbose=True,
)

In [7]:
cp_model = decompose_model(
    model,
    conv_rank=1200,
    fc_rank=1800,
    loss_fn=loss_fn,
    train_fn=train_fn,
    trial_rank=5,
    layer_size_regularization=0.6,
    linear_decomp_type="svd",
    freeze_decomposed=False,
    verbose=True,
)

Computing losses for conv layers
Processing module features.0


Validation: 100%|██████████| 50/50 [00:08<00:00,  6.01it/s, loss=8.36]


Processing module features.3


Validation: 100%|██████████| 50/50 [00:07<00:00,  6.56it/s, loss=7.48]


Processing module features.6


...

Decomposing classifier.1 with rank 786
Training model with classifier.1 decomposed


Training: 100%|██████████| 500/500 [00:55<00:00,  9.03it/s, acc_1=0.465, acc_5=0.723, loss=2.4] 


Decomposing classifier.4 with rank 602
Training model with classifier.4 decomposed


Training: 100%|██████████| 500/500 [00:50<00:00,  9.81it/s, acc_1=0.477, acc_5=0.738, loss=2.24]


Decomposing classifier.6 with rank 412


In [8]:
def evaluate(
    model: nn.Module, dataloader: DataLoader, device: Union[str, torch.device]
):
    model.eval()
    model.to(device)
    accs = []
    accs_5 = []
    loop = tqdm(dataloader, desc="Evaluation")
    for x, y in loop:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            y_pred = model(x)
            accs.append(acc_1(y_pred, y))
            accs_5.append(acc_5(y_pred, y))
            loop.set_postfix(acc=mean(accs), acc_5=mean(accs_5))
    return mean(accs), mean(accs_5)

In [9]:
accuracies = evaluate(cp_model, val_loader, "cuda")
print(f"Top-1 Accuracy: {accuracies[0]:.3f} | Top-5 Accuracy: {accuracies[1]:.3f}")

Evaluation:   0%|          | 0/157 [00:00<?, ?it/s]

Evaluation: 100%|██████████| 157/157 [00:32<00:00,  4.78it/s, acc=0.474, acc_5=0.737]

Top-1 Accuracy: 0.474 | Top-5 Accuracy: 0.737


In [10]:
ref_model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
cp_model.cpu()
ref_model.cuda()
accuracies_ref = evaluate(ref_model, val_loader, "cuda")
print(f"Top-1 Accuracy: {accuracies_ref[0]:.3f} | Top-5 Accuracy: {accuracies_ref[1]:.3f}")

Evaluation: 100%|██████████| 157/157 [00:25<00:00,  6.08it/s, acc=0.562, acc_5=0.785]

Top-1 Accuracy: 0.562 | Top-5 Accuracy: 0.785


In [11]:
def get_size_ratio(model1: nn.Module, model2: nn.Module) -> float:
    size1 = sum(p.numel() for p in model1.parameters())
    size2 = sum(p.numel() for p in model2.parameters())
    return size1 / size2

size_ratio = get_size_ratio(cp_model, ref_model)
print(f"Size Ratio: {size_ratio:.3f}")

Size Ratio: 0.296
